In [1]:
import chromadb
from typing_extensions import Annotated

import autogen
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [2]:
api_key = "<your-api-key>"
azure_endpoint = "<your-endpoint>"
api_version = "<your-api-version>"

llm_config = {"api_key": api_key, "api_version":api_version, "azure_endpoint":azure_endpoint, "model":"<your-model", "api_type":"azure", "cache_seed": None}

In [3]:
def termination_msg(x):
    return isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

In [4]:
topics_generator = autogen.UserProxyAgent(
    name="TopicsGenerator1",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    code_execution_config=False,  # we don't want to execute code in this case.
    description="An Agent which generates topics for the article that it is asked to write",
)

In [5]:
examples_adder = AssistantAgent(
    name="ExamplesAdder",
    is_termination_msg=termination_msg,
    system_message="You read the json having an article written. Your job is to add more examples in that json. Add a new key Examples and then add atleast 4 examples to it. ",
    llm_config=llm_config,
    description="Adding more examples",
)

In [6]:
advanced_topic = AssistantAgent(
    name="AdvancedTopic",
    is_termination_msg=termination_msg,
    system_message="You read the json having an article written. Your job is to add a new key which talks about advanced topics related to the article. Give final response in the same JSON format and then reply with TERMINATE",
    llm_config=llm_config,
    description="Adding a section on Advanced Topics",
)

In [7]:
article_writer = autogen.AssistantAgent(
    name="ArticleWriter1",
    is_termination_msg=termination_msg,
    system_message="""You are a technical blog writer, you write an article on the topics provided by TopicsGenerator. Each topic should be written with a simple example. Make sure the final response that you give should be in JSON format so that it can be rendered in the UI. It should follow an example structure given below:
     \{
    "Title":"Unraveling Numpy Broadcasting With Clear Examples",
    "Body":\[
        \{
            "Content": "In the vibrant world of Python programming, Numpy stands as a fundamental library that empowers its users with extensive mathematical capabilities. Among these wonderful features of Numpy, a little appreciated yet pivotal one is 'Broadcasting'. This article will vividly explain and illustrate this fairly perplexing aspect of Numpy."
        \},
        \{
            "Heading": "What is Numpy Broadcasting?",
            "Content": "Numpy broadcasting is a powerful feature that allows us to perform arithmetic operations on arrays of different shapes. This could be between a scalar and an array, or between arrays of different shapes, under certain conditions."
        \},
        \{
            "Content": "The concept behind broadcasting pivots on two rules:",
            "List": \[
                \{
                    "ListItem": "If the arrays do not have the same rank, then a 1 will be pre-pended to the smaller ranking array’s shape."
                \}
            \]
        \},
        \{
            "Heading": "Python example of Numpy Broadcasting",
            "Content": "Let's illustrate this concept with a simple Python example.",
            "Code": \[
                \{
                    "CodeBlock": "import numpy as np\nA = np.array([1, 2, 3])\nB = np.array([2, 2, 2])\nC = A * B\nprint(C)"
                \},
                \{
                    "Content": "In this example, we're multiplying two arrays of identical shape, so broadcasting isn't necessary. However, observe what happens when we alter the second array:",
                    "CodeBlock": "B = 2\nC = A * B\nprint(C)"
                \}
            \]
        \}
    \]
\}""",
    llm_config=llm_config,
    description="technical blog writer who writes article on the list of topics provided",
)

In [10]:
def _reset_agents():
    topics_generator.reset()
    article_writer.reset()
    examples_adder.reset()
    advanced_topic.reset()
    topics_generator.clear_history()
    article_writer.clear_history()
    examples_adder.clear_history()
    advanced_topic.clear_history()

In [11]:
_reset_agents()

In [12]:
import shutil
def create_article(PROBLEM):
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[topics_generator, article_writer, examples_adder, advanced_topic],
        messages=[],
        max_round=4,
        speaker_selection_method="round_robin",
        allow_repeat_speaker=False,
        enable_clear_history=True
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config, max_consecutive_auto_reply=1)
    topics_generator.initiate_chat(
        manager,
        message=PROBLEM,
    )

In [15]:
import json
topics = ["Measures of Central Tendency (Mean, Median, Mode)", "Measures of Dispersion (Variance, Standard Deviation, Range)"]
final_json = []
for i, topic in enumerate(topics):
    PROBLEM = f"Write an article similar to a blog on {topic} with respect to Python and Machine Learning"
    create_article(PROBLEM)
    cleaned_json_string = topics_generator.last_message()["content"].replace("TERMINATE", "").strip()
    data = json.loads(cleaned_json_string)
    final_json.append(data)
    print(data)

TopicsGenerator1 (to chat_manager):

Write an article similar to a blog on Measures of Central Tendency (Mean, Median, Mode) with respect to Python and Machine Learning

--------------------------------------------------------------------------------
ArticleWriter1 (to chat_manager):

{
    "Title":"Measures of Central Tendency in Python and Machine Learning",
    "Body":[
        {
            "Content": "Measures of central tendency are statistical measures that represent the center or average of a dataset. In Python and machine learning, these measures, namely mean, median, and mode, play a vital role in understanding data distribution and making informed decisions. In this article, we will explore these measures and how they are used in Python and machine learning."
        },
        {
            "Heading": "Mean",
            "Content": "The mean is the sum of all values in a dataset divided by the total number of values. In Python, calculating the mean is a breeze using librari

In [94]:
{**{}, **{1:4, 2:5}}

{1: 4, 2: 5}

In [17]:
final = {}
for f in final_json:
    final = {**final, **f}

In [18]:
final

{'Title': 'Understanding Measures of Dispersion in Python and Machine Learning',
 'Body': [{'Content': "When dealing with data, it's important to not only understand the central tendency but also the spread or dispersion of the data points. Measures of dispersion provide insights into how data points are distributed around the mean. In this article, we will explore three common measures of dispersion: variance, standard deviation, and range, and discuss their significance in the context of Python and machine learning."},
  {'Heading': 'Variance',
   'Content': 'Variance is a measure of how spread out the data points are in a dataset. It quantifies the average squared deviation from the mean. In Python, the NumPy library provides efficient methods to calculate variance. For example:'},
  {'Code': [{'CodeBlock': 'import numpy as np\n\n# Create a sample dataset\ndata = np.array([5, 7, 8, 11, 15])\n\n# Calculate variance\nvariance = np.var(data)\n\nprint(variance)'}]},
  {'Heading': 'Stand